In [1]:
f = open('/mnt/e/Temp/yslzm/com.zulong.yslzm.ios-1.5.3-1552333447-855749158/Payload/Azure.app/StreamingAssets/res_base/package_dec/data/achievement_essence.bcfg', 'rb')
bcfgContent = f.read()

In [8]:
import io
class IOHelper(io.BytesIO):
    def readS16(self):
        return int.from_bytes(self.read(2), 'big', signed=True)
    
    def readU16(self):
        return int.from_bytes(self.read(2), 'big')
    
    def readS32(self):
        return int.from_bytes(self.read(4), 'big', signed=True)
    
    def readU32(self):
        return int.from_bytes(self.read(4), 'big')
    
    def eof(self):
        if self.tell() >= len(self.getvalue()):
            return True
    
    def read(self, l):
        ret = super().read(l)
        assert len(ret) == l
        return ret

    def readVarint(self):
        b0 = self.read(1)[0]
        if b0 & 0x80 == 0: # 0XXXXXXX
            return b0 & 0x7f
        elif b0 & 0x40 == 0: # 10XXXXXX XXXXXXXX
            return int.from_bytes(bytes([b0 & 0x3f]) + self.read(1), 'big')
        elif b0 & 0x20 == 0: # 110XXXXX XXXXXXXX XXXXXXXX XXXXXXXX
            return int.from_bytes(bytes([b0 & 0x1f]) + self.read(2), 'big')
        elif b0 & 0x10 == 0: # 1110XXXX XXXXXXXX XXXXXXXX XXXXXXXX
            return int.from_bytes(bytes([b0 & 0xf]) + self.read(3), 'big')
        else: # 1111XXXX XXXXXXXX XXXXXXXX XXXXXXXX
            return int.from_bytes(read(4), 'big')


In [25]:
import logging
from typing import *
import io
import dataclasses
import struct

logging.getLogger().setLevel(logging.DEBUG)
logger = logging.getLogger('bcfg_parser')

@dataclasses.dataclass(init=False)
class BCFGHeader:
    '''
    struct BCFGHeader {
        char magic[2];
        uint16 defNum;
        uint32 rowNum;
        uint32 oriDataSize;
        uint16 tailSize;
        uint16 chunkNum;
        uint32 tableId;
    } bcfgHeader;
    '''
    magic: bytes
    defNum: int
    rowNum: int
    oriDataSize: int
    tailSize: int
    chunkNum: int
    tableId: int

@dataclasses.dataclass(init=False)
class BCFGDef:
    '''
    struct Def {
        varint defType;
        varint namelen;
        char name[VarintValue(namelen)];
        varint varnum;
        struct VarInfo {
            varint vartype;
            varint varNameLen;
            char varName[VarintValue(varNameLen)];
        } vars[VarintValue(varnum)] <optimize=false>;
    } defs[bcfgHeader.defNum] <optimize=false>;
    '''
    defType: int
    defName: str
    @dataclasses.dataclass(init=False)
    class BCFGVar:
        varType: int
        varName: str
    defVars: List[BCFGVar]

@dataclasses.dataclass(init=False)
class BCFGRow:
    rowId: int
    rowOffset: int
    rowData: Dict

        
class BCFGParser:
    def __init__(self, bcfgContent):
        self.bcfgContent = bcfgContent
        self.bcfgReader = IOHelper(self.bcfgContent)
    
    def parseBcfgHeader(self):
        logger.info("parseBcfgHeader!")
        
        read = self.bcfgReader.read
        readU16 = self.bcfgReader.readU16
        readU32 = self.bcfgReader.readU32
        
        self.bcfgHeader = BCFGHeader()
        self.bcfgHeader.magic = read(2)
        assert self.bcfgHeader.magic == b'cx'
        self.bcfgHeader.defNum = readU16()
        self.bcfgHeader.rowNum = readU32()
        self.bcfgHeader.oriDataSize = readU32()
        self.bcfgHeader.tailSize = readU16()
        self.bcfgHeader.chunkNum = readU16()
        self.bcfgHeader.tableId = readU32()
        self.oriData = read(self.bcfgHeader.oriDataSize)
        self.oriDataIO = IOHelper(self.oriData)
    
    def parseBcfgDef(self):
        logger.info("parseBcfgDef!")
        read = self.bcfgReader.read
        readU16 = self.bcfgReader.readU16
        readU32 = self.bcfgReader.readU32
        readVarint = self.bcfgReader.readVarint
        
        self.defs = {}
        for i in range(self.bcfgHeader.defNum):
            logger.info("parseBcfgDef, def %d", i)
            curDef = BCFGDef()
            curDef.defType = readVarint()
            curDef.defName = read(readVarint()).decode()
            varNum = readVarint()
            curDef.defVars = []
            for j in range(varNum):
                curVar = BCFGDef.BCFGVar()
                curVar.varType = readVarint()
                curVar.varName = read(readVarint()).decode()
                curDef.defVars.append(curVar)
            self.defs[curDef.defType] = curDef
    
    def parseRows(self):
        read = self.bcfgReader.read
        readU16 = self.bcfgReader.readU16
        readU32 = self.bcfgReader.readU32
        readVarint = self.bcfgReader.readVarint
        
        for i in range(self.bcfgHeader.rowNum):
            curRow = BCFGRow()
            curRow.rowId = readVarint()
            curRow.rowOffset = readVarint()
            
            def parseBeanType(reader, beanType):
                logger.info("parseBeanType %x", beanType)
                ret = {}
                curDef = self.defs[beanType]
                for defVar in curDef.defVars:
                    ret[defVar.varName] = parseType(reader, defVar.varType)
                return ret
            
            def parseListType(reader, vartype):
                logger.info("parseListType %x", vartype)
                assert (vartype & 0xF) == 0xF
                elemVarType = vartype & ~0xF
                ret = []
                while not reader.eof():
                    ret.append(parseType(reader, elemVarType))
                return ret
                
            def parseType(reader, vartype):
                logger.info("parseType %x", vartype)
                if vartype == 0x10: # boolean
                    return int.from_bytes(reader.read(1), 'big')
                elif vartype == 0x20: # int16
                    return reader.readS16()
                elif vartype == 0x30: # uint16
                    return reader.readU16()
                elif vartype == 0x40: # int32
                    return reader.readS32()
                elif vartype == 0x50: # uint32
                    return reader.readU32()
                elif vartype == 0x80: # float
                    return struct.unpack('>f', reader.read(4))[0]
                elif vartype == 0x90: # string
                    return reader.read(reader.readVarint()).decode()
                else:
                    if (vartype & 0xF) == 0xF: # list
                        listData = reader.read(reader.readVarint())
                        return parseListType(IOHelper(listData), vartype)
                    elif (vartype & 0xF0) == 0xA0: # table
                        listData = reader.read(reader.readVarint())
                        return parseBeanType(IOHelper(listData), vartype)
                    
            self.oriDataIO.seek(curRow.rowOffset)
            rowData = parseBeanType(self.oriDataIO, 0)
            logger.info("Get row: %s", )
            break
    
    def doParse(self):
        self.parseBcfgHeader()
        self.parseBcfgDef()
        self.parseRows()

p = BCFGParser(bcfgContent)
p.doParse()

AssertionError: 